# hmmlearn

In [1]:
from hmmlearn import hmm
import numpy as np

In [2]:
model = hmm.GaussianHMM(n_components=3, covariance_type="full")
model.startprob_ = np.array([0.1, 0.8, 0.1])
model.transmat_ = np.array([[0.8, 0.15, 0.05],
                            [0.05, 0.9, 0.05],
                            [0.05, 0.15, 0.8]])
model.means_ = np.array([[0.0], [3.0], [5.0]])
model.covars_ = np.tile(np.identity(1), (3, 1, 1))
X, Z = model.sample(100)
print(Z)
print(model.predict(X))
print(Z==model.predict(X))

[1 1 1 2 2 2 1 1 1 2 2 2 2 2 2 2 2 2 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1
 1 1 1 1 2 2 1 1 1 1 1 1 1 1 1 2 2 2 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 1
 0 1 1 1 1 1 2 2 2 2 2 2 2 0 0 0 0 0 0 0 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1
 1 1 1 2 2 2 1 1 1 1 1 1 1 1 1 2 2 2 2 2 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 1
 1 1 1 1 1 1 2 2 2 2 2 2 2 0 0 0 0 0 0 0 1 1 1 1 1 1]
[ True  True  True False False False  True  True  True False False False
 False  True  True  True  True  True False False  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True False  True  True  True  True  True  True  True
  True  True  True  True  True  True  True False False  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  

/Users/chenyl/.virtualenvs/python-lib-test/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function sample_gaussian is deprecated; The function sample_gaussian is deprecated in 0.18 and will be removed in 0.20. Use numpy.random.multivariate_normal instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/chenyl/.virtualenvs/python-lib-test/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function sample_gaussian is deprecated; The function sample_gaussian is deprecated in 0.18 and will be removed in 0.20. Use numpy.random.multivariate_normal instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/chenyl/.virtualenvs/python-lib-test/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function sample_gaussian is deprecated; The function sample_gaussian is deprecated in 0.18 and will be removed in 0.20. Use numpy.random.multivariate_normal instead.
  warnings.warn(msg, category=